# Plot Points

The goal of this notebook is to simplify a workflow of getting points off of a datasheet for any purpose (regression, for example).

To do this, we will use an interactive `matplotlib` session where you will click on the plot to place points which will be translated from local positions on the image to values using information about the axes that you will provide.

Run this block to acquire the necessary imports. We also specify `matplotlib` to use a backend that is capable of doing what we need, which is not the default when using Jupyter Notebooks.

In [1]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

In order for this program to translate from the positions of the points in abstract local space on the image to the real values we want out of the datasheet, we need to provide information to perform this translation.
When you begin the interactive session, the first step will be to place three points. The first should go at the origin, the second at the top of the $y$–axis, and the third at the rightmost (end) of the $x$–axis. Hopefully it is evident how these points will be used as reference points for the data points you place afterwards. Nonetheless, to use those points to translate between local positions on the image to actual values, you'll also need to provide information about what values are represented at each point on each axis.

For example, if I had a datasheet where I had a graph of time vs position, the $x$–axis may be time between $0s$ and $10s$, and the $y$–axis may be position between $10m$ and $100m$. Thus, I'd put a point at the origin $(0, 10)$, then at the top of the $y$–axis at $(0, 100)$ and then at the end of the $x$–axis at $(10, 10)$. Before that, I would've set `x_begin=0`, `x_end=10`, `y_begin=10`, and `y_end=100`.

You'll also need to set the following environment variables.
1. `path`: the relative path from this directory to your image.
2. `num_points`: the number of points that you'd like to extract (excluding the guide points)

In [2]:
# -- ALL OF THESE MUST BE SET FOR DATA TO BE MEANINGFUL --
path: str = 'data_analysis/data/battery_cell_datasheet.png'
num_points: int = 33
x_begin: float = 0.0
x_end: float = 3500.0
y_begin: float = 2.0
y_end: float = 4.5

> Make sure that you actually ran the block above after setting the variables, or they won't actually be loaded in the environment!

Finally, run the block below to begin the interactive session. Remember to place the three guide points first, and then place up to `num_points` points on the plot wherever you'd like to extract them.
Close the window whenever you're done, and the points will be ready when you come back to this notebook.

In [3]:
img = plt.imread(path)
plt.imshow(img)
plt.axis('off')

Point = list[float, float]
# Allow interactive point selection
points: list[Point] = plt.ginput(n=num_points + 3, show_clicks=True, timeout=10000)

# Display selected points
for point in points:
    plt.plot(point[0], point[1], 'ro')  # Plot selected points as red dots

# Show the plot
plt.show()

Next, run the block below to translate the points in raw local space to the real values using the information you provided.

In [4]:
# Assign guide points
origin_point: Point = points[0]
y_top: Point = points[1]
x_right: Point = points[2]

# Get ranges
y_range_real: float = y_end - y_begin
x_range_real: float = x_end - x_begin

x_range_local: float = x_right[0] - origin_point[0]
y_range_local: float = y_top[1] - origin_point[1]

# Get conversion factors
x_local_to_real: float = x_range_real / x_range_local
y_local_to_real: float = y_range_real / y_range_local

real_points: list[Point] = []

for point in points:
    # Translate
    real_x: float = point[0] - origin_point[0]
    real_y: float = point[1] - origin_point[1]

    # Scale
    real_x *= x_local_to_real
    real_y *= y_local_to_real

    # Localize
    real_x += x_begin
    real_y += y_begin

    real_points.append([real_x, real_y])

# Output result
print(real_points)

[[0.0, 2.0], [-6.294964028776892, 4.5], [3500.0, 4.50796178343949], [-18.884892086330854, 2.3582802547770703], [50.35971223021584, 2.4856687898089174], [258.0935251798561, 2.4856687898089174], [453.2374100719426, 2.5573248407643314], [686.1510791366908, 2.6687898089171975], [925.3597122302157, 2.71656050955414], [1561.1510791366902, 2.8359872611464967], [1919.964028776979, 3.1863057324840764], [2990.1079136690646, 3.5445859872611463], [3008.9928057553952, 3.5525477707006368]]


You may find it suitable to customize the output style for your use case.